# Import dataset from CSV file (Raw data)

We read the Titanic dataset by using spark a CSV file here and create a pyspark dataframe accordingly.

In [0]:
# File location and type
file_location = "/FileStore/tables/train-1.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
sdf = (spark.read.format(file_type) 
  .option("inferSchema", infer_schema) 
  .option("header", first_row_is_header) 
  .option("sep", delimiter) 
  .load(file_location))

# Data preparation work

## Working on Ticket

First we need to check if all passengers have a Ticket information filled out. If we have Null values we'll need to fill those records with X that means no data.

In [0]:
from pyspark.sql.functions import when, col

sdf = sdf.fillna({'Ticket': 'X'})
# Add a new column noTicket that will contain 1 if there was no ticket, 0 else.
sdf = sdf.withColumn("noTicket", when(col("Ticket") == 'X', 1).otherwise(0))
# Count the number of passengers without ticket
value_counts = sdf.groupBy("noTicket").count()
display(value_counts)

noTicket,count
0,891


Great, everyone had a Ticket.  
Now let's see if there are duplicates in the passenger's Tickets.

In [0]:
from pyspark.sql.functions import count

# Count the number of identical Tickets
ticket_counts = sdf.groupBy("Ticket").agg(count("Ticket").alias("passenger_count"))
# Only keep the duplicates (if any) and put them in a new dataframe
duplicate_tickets = ticket_counts.filter(col("passenger_count") > 1)
display(duplicate_tickets)

Ticket,passenger_count
367230,2
P/PP 3381,2
363291,3
26360,2
19877,2
C.A. 2673,2
PC 17604,2
C.A. 2315,2
31027,2
1601,7


We can also get all the passengers infos for those duplicates:

In [0]:
passengers_with_the_same_ticket = sdf.join(duplicate_tickets, 
                                           on="Ticket", 
                                           how="inner")
display(passengers_with_the_same_ticket)

Ticket,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,passenger_count
113803,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,53.1,C123,S,2
349909,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,21.075,null,S,4
347742,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,11.1333,null,S,3
237736,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,30.0708,null,C,2
PP 9549,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,16.7,G6,S,2
347082,14,0,3,"Andersson, Mr. Anders Johan",male,39,1,5,31.275,null,S,7
382652,17,0,3,"Rice, Master. Eugene",male,2,4,1,29.125,null,Q,5
239865,21,0,2,"Fynney, Mr. Joseph J",male,35,0,0,26,null,S,2
349909,25,0,3,"Palsson, Miss. Torborg Danira",female,8,3,1,21.075,null,S,4
347077,26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)",female,38,1,5,31.3875,null,S,4


We have 344 passengers that share the same ticket number. That's a information we need to add in our current dataset, let's add the passenger_count column into the initial dataset:

In [0]:
from pyspark.sql.functions import when, col, count

ticket_counts = sdf.groupBy("Ticket").agg(count("Ticket").alias("ticket_sharing_number"))
sdf = (sdf.join(ticket_counts, on="Ticket", how="left").
                withColumn("sharing_Ticket", 
                           when(col("ticket_sharing_number") > 1, 1).otherwise(0)))
display(sdf)

Ticket,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,ticket_sharing_number,sharing_Ticket
A/5 21171,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,7.25,null,S,1,0
PC 17599,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,71.2833,C85,C,1,0
STON/O2. 3101282,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,7.925,null,S,1,0
113803,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,53.1,C123,S,2,1
373450,5,0,3,"Allen, Mr. William Henry",male,35,0,0,8.05,null,S,1,0
330877,6,0,3,"Moran, Mr. James",male,null,0,0,8.4583,null,Q,1,0
17463,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,51.8625,E46,S,1,0
349909,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,21.075,null,S,4,1
347742,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,11.1333,null,S,3,1
237736,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,30.0708,null,C,2,1


## Working on Fare

As we have people sharing the same ticket number we now can gues we should also recalcultate the Fare, and so calculate the Ticket unit price:

In [0]:
from pyspark.sql.functions import when, col

# Add the calculated unit Fare = Fare / sharing_Ticket
sdf = sdf.withColumn('unit_Fare', when(col("sharing_Ticket") == 1, col("Fare") / 2).otherwise(col("Fare")))
     
display(sdf)                               

Ticket,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,ticket_sharing_number,sharing_Ticket,unit_Fare
A/5 21171,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,7.25,null,S,1,0,7.25
PC 17599,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,71.2833,C85,C,1,0,71.2833
STON/O2. 3101282,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,7.925,null,S,1,0,7.925
113803,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,53.1,C123,S,2,1,26.55
373450,5,0,3,"Allen, Mr. William Henry",male,35,0,0,8.05,null,S,1,0,8.05
330877,6,0,3,"Moran, Mr. James",male,null,0,0,8.4583,null,Q,1,0,8.4583
17463,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,51.8625,E46,S,1,0,51.8625
349909,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,21.075,null,S,4,1,10.5375
347742,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,11.1333,null,S,3,1,5.56665
237736,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,30.0708,null,C,2,1,15.0354


## Working on Full Name

The column Name contains several information like first name, last name and title. We'll leverage regular expressions to extract these data from this field:

In [0]:
from pyspark.sql.functions import regexp_extract

# Extract the family name (everything before the comma)
sdf = sdf.withColumn("Family_Name", regexp_extract(col("Name"), r"^([^,]+)", 1))

# Extract the title (word between the comma and period)
sdf = sdf.withColumn("Title", regexp_extract(col("Name"), r",\s*([^\.]+)\.", 1))

# Extract the first name (everything after the title and period)
sdf = sdf.withColumn("First_Name", regexp_extract(col("Name"), r"\.\s*(.+)$", 1))

display(sdf)

Ticket,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,ticket_sharing_number,sharing_Ticket,unit_Fare,Family_Name,Title,First_Name
A/5 21171,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,7.25,null,S,1,0,7.25,Braund,Mr,Owen Harris
PC 17599,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,71.2833,C85,C,1,0,71.2833,Cumings,Mrs,John Bradley (Florence Briggs Thayer)
STON/O2. 3101282,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,7.925,null,S,1,0,7.925,Heikkinen,Miss,Laina
113803,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,53.1,C123,S,2,1,26.55,Futrelle,Mrs,Jacques Heath (Lily May Peel)
373450,5,0,3,"Allen, Mr. William Henry",male,35,0,0,8.05,null,S,1,0,8.05,Allen,Mr,William Henry
330877,6,0,3,"Moran, Mr. James",male,null,0,0,8.4583,null,Q,1,0,8.4583,Moran,Mr,James
17463,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,51.8625,E46,S,1,0,51.8625,McCarthy,Mr,Timothy J
349909,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,21.075,null,S,4,1,10.5375,Palsson,Master,Gosta Leonard
347742,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,11.1333,null,S,3,1,5.56665,Johnson,Mrs,Oscar W (Elisabeth Vilhelmina Berg)
237736,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,30.0708,null,C,2,1,15.0354,Nasser,Mrs,Nicholas (Adele Achem)


## Working on Age

It could also be interresting to build out age categories by using the Age provided.

In [0]:
from pyspark.sql.functions import when

display(sdf.groupBy(
    when(col("Age").isNull(), "NULL").otherwise(col("Age")).alias("Age")
).count())

Age,count
20.5,1
7,3
51,7
0.75,2
54,8
15,5
11,4
29,20
42,13
64,2


That's weird several ages are float numbers ! maybe it's a data quality issue, but at this stage we'll just convert thi column in integer.

In [0]:
from pyspark.sql.functions import col

# Convert Age column to integer
sdf = sdf.withColumn("Age", col("Age").cast("int"))

Now we create the age category column:

In [0]:
from pyspark.sql.functions import when, col

# Define age categories based on the given ranges
sdf = sdf.withColumn(
    "AgeCategory",
    when(col("Age").isNull(), "Unknown")  # New category for NULL values
    .when(col("Age") < 2, "babies")  # Age less than 2
    .when((col("Age") >= 2) & (col("Age") < 18), "young")  # Age between 2 and 18
    .when((col("Age") >= 18) & (col("Age") < 65), "adult")  # Age between 18 and 65
    .otherwise("old")  # Age 65 or older
)
display(sdf)

Ticket,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,ticket_sharing_number,sharing_Ticket,unit_Fare,Family_Name,Title,First_Name,AgeCategory
A/5 21171,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,7.25,null,S,1,0,7.25,Braund,Mr,Owen Harris,adult
PC 17599,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,71.2833,C85,C,1,0,71.2833,Cumings,Mrs,John Bradley (Florence Briggs Thayer),adult
STON/O2. 3101282,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,7.925,null,S,1,0,7.925,Heikkinen,Miss,Laina,adult
113803,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,53.1,C123,S,2,1,26.55,Futrelle,Mrs,Jacques Heath (Lily May Peel),adult
373450,5,0,3,"Allen, Mr. William Henry",male,35,0,0,8.05,null,S,1,0,8.05,Allen,Mr,William Henry,adult
330877,6,0,3,"Moran, Mr. James",male,null,0,0,8.4583,null,Q,1,0,8.4583,Moran,Mr,James,Unknown
17463,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,51.8625,E46,S,1,0,51.8625,McCarthy,Mr,Timothy J,adult
349909,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,21.075,null,S,4,1,10.5375,Palsson,Master,Gosta Leonard,young
347742,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,11.1333,null,S,3,1,5.56665,Johnson,Mrs,Oscar W (Elisabeth Vilhelmina Berg),adult
237736,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,30.0708,null,C,2,1,15.0354,Nasser,Mrs,Nicholas (Adele Achem),young


In [0]:
display(sdf.groupBy("AgeCategory").count())

AgeCategory,count
old,11
Unknown,177
babies,14
adult,590
young,99


## Adding another title (not related to the gender)

In [0]:
from pyspark.sql.functions import when, col

vip = ['Don','Sir', 'Major', 'Col', 'Jonkheer', 'Dr']
women_and_kids = ['Miss', 'Mrs', 'Lady', 'Mlle', 'the Countess', 'Ms', 'Mme', 'Dona', 'Master']
sdf = sdf.withColumn("vipTitle",
                        when(col("Title").isin(women_and_kids), "women_and_kids")
                        .when(col("Title").isin(vip), "vip")
                        .otherwise("Others"))
display(sdf)

Ticket,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,noTicket,ticket_sharing_number,sharing_Ticket,unit_Fare,Family_Name,Title,First_Name,AgeCategory,vipTitle
A/5 21171,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,7.25,null,S,0,1,0,7.25,Braund,Mr,Owen Harris,adult,Others
PC 17599,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,71.2833,C85,C,0,1,0,71.2833,Cumings,Mrs,John Bradley (Florence Briggs Thayer),adult,women_and_kids
STON/O2. 3101282,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,7.925,null,S,0,1,0,7.925,Heikkinen,Miss,Laina,adult,women_and_kids
113803,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,53.1,C123,S,0,2,1,26.55,Futrelle,Mrs,Jacques Heath (Lily May Peel),adult,women_and_kids
373450,5,0,3,"Allen, Mr. William Henry",male,35,0,0,8.05,null,S,0,1,0,8.05,Allen,Mr,William Henry,adult,Others
330877,6,0,3,"Moran, Mr. James",male,null,0,0,8.4583,null,Q,0,1,0,8.4583,Moran,Mr,James,old,Others
17463,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,51.8625,E46,S,0,1,0,51.8625,McCarthy,Mr,Timothy J,adult,Others
349909,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,21.075,null,S,0,4,1,10.5375,Palsson,Master,Gosta Leonard,young,women_and_kids
347742,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,11.1333,null,S,0,3,1,5.56665,Johnson,Mrs,Oscar W (Elisabeth Vilhelmina Berg),adult,women_and_kids
237736,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,30.0708,null,C,0,2,1,15.0354,Nasser,Mrs,Nicholas (Adele Achem),young,women_and_kids


Let's see how many passengers by category we have:

In [0]:
display(sdf.groupBy("vipTitle").count())

vipTitle,count
women_and_kids,353
Others,524
vip,14


## Fill Null values

In [0]:
sdf = sdf.fillna({"Age": 0, "Embarked": "X", "cabin": "X", 
                    "Title": "X", "Family_Name": "X", "First_Name": "X", 
                    "AgeCategory": "X", "vipTitle": "X", "unit_Fare": 0})

## Final touches ...

In [0]:
sdf = sdf.drop(*["Name", "Fare"])


In [0]:
display(sdf)

Ticket,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked,noTicket,ticket_sharing_number,sharing_Ticket,unit_Fare,Family_Name,Title,First_Name,AgeCategory,vipTitle
A/5 21171,1,0,3,male,22,1,0,X,S,0,1,0,7.25,Braund,Mr,Owen Harris,adult,Others
PC 17599,2,1,1,female,38,1,0,C85,C,0,1,0,71.2833,Cumings,Mrs,John Bradley (Florence Briggs Thayer),adult,women_and_kids
STON/O2. 3101282,3,1,3,female,26,0,0,X,S,0,1,0,7.925,Heikkinen,Miss,Laina,adult,women_and_kids
113803,4,1,1,female,35,1,0,C123,S,0,2,1,26.55,Futrelle,Mrs,Jacques Heath (Lily May Peel),adult,women_and_kids
373450,5,0,3,male,35,0,0,X,S,0,1,0,8.05,Allen,Mr,William Henry,adult,Others
330877,6,0,3,male,0,0,0,X,Q,0,1,0,8.4583,Moran,Mr,James,old,Others
17463,7,0,1,male,54,0,0,E46,S,0,1,0,51.8625,McCarthy,Mr,Timothy J,adult,Others
349909,8,0,3,male,2,3,1,X,S,0,4,1,10.5375,Palsson,Master,Gosta Leonard,young,women_and_kids
347742,9,1,3,female,27,0,2,X,S,0,3,1,5.56665,Johnson,Mrs,Oscar W (Elisabeth Vilhelmina Berg),adult,women_and_kids
237736,10,1,2,female,14,1,0,X,C,0,2,1,15.0354,Nasser,Mrs,Nicholas (Adele Achem),young,women_and_kids
